In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import csv



In [2]:
# Setup Firefox options
firefox_options = Options()
# firefox_options.add_argument("--headless")  # ensure GUI is off

# Set the Firefox driver (update the path if needed)
webdriver_service = Service('geckodriver.exe')
driver = webdriver.Firefox(service=webdriver_service, options=firefox_options)


In [3]:
url = 'https://finance.yahoo.com/quote/TSLA?p=TSLA'
driver.get(url)


In [4]:
stock = url.split('/')[-1].split('?')[0]

print(stock)


TSLA


## Download historical Data

In [103]:
# click on historical data tab 
try:
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[5]/a").click()
except:
    driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[5]/a").click()
    
# click on time period tab 
time.sleep(1)
try:
    dates = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div[1]/div/div/div/span").text
except:
    dates = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div[1]/div/div/div").text
                                       
# Extract the end date from the string
end_date_str = dates.split('-')[1].strip()

# Convert the end date string to a datetime object
end_date = datetime.strptime(end_date_str, "%b %d, %Y")

# Subtract 5 years from the end date
five_years_before = end_date - timedelta(days=5*365)

# Convert the new date back to a string
five_years_before_str = five_years_before.strftime("%d / %m / %Y")

print(five_years_before_str)

driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div[1]/div/div/div').click()
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div[1]/div/div/div[2]/div/ul[2]/li[3]/button').click()
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/button').click()
time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[2]/span[2]/a').click()


NoSuchElementException: Message: Unable to locate element: /html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[5]/a; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


## Get Company Financials

In [106]:
try:
    driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
except:
    try:
        driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[7]/a').click()
    except:                           
        driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
    

### Income Statement

In [107]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/button').click()


In [111]:
try:
    table_header = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div')
except:
    table_header = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[3]/div[1]/div/div[1]/div')
table_html = table_header.get_attribute('innerHTML')
soup = BeautifulSoup(table_html, 'html.parser')
spans = soup.find_all('span')

csv_data = []
row = []
for span in spans:
    row.append(span.text)
    print(span.text)

csv_data.append(row)

try:
    row = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]')
except:
    row = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[3]/div[1]/div/div[2]')
row_html = row.get_attribute('innerHTML')
# <div class="D(tbr) fi-row Bgc($hoverBgColor):h"><div class="D(tbc) Ta(start) Pend(15px)--mv2 Pend(10px) Bxz(bb) Py(8px) Bdends(s) Bdbs(s) Bdstarts(s) Bdstartw(1px) Bdbw(1px) Bdendw(1px) Bdc($seperatorColor) Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pstart(15px)--mv2 Pstart(10px)"><div class="D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined" title="Total Revenue"><button aria-label="Total Revenue" class="P(0) M(0) Va(m) Bd(0) Fz(s) Mend(2px) tgglBtn"><svg class="H(16px) Fill($primaryColor) Stk($primaryColor) tgglBtn:h_Fill($linkColor) tgglBtn:h_Stk($linkColor) Cur(p)" width="16" style="stroke-width:0;vertical-align:bottom" height="16" viewBox="0 0 48 48" data-icon="caret-down"><path d="M24.21 33.173l12.727-12.728c.78-.78.78-2.048 0-2.828-.78-.78-2.047-.78-2.828 0l-9.9 9.9-9.9-9.9c-.78-.78-2.047-.78-2.827 0-.78.78-.78 2.047 0 2.828L24.21 33.173z"></path></svg></button><span class="Va(m)">Total Revenue</span></div><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>94,028,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>81,462,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>53,823,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>31,536,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>24,578,000</span></div></div><div><div class="rw-expnded" data-test="fin-row"><div class="D(tbr) fi-row Bgc($hoverBgColor):h"><div class="D(tbc) Ta(start) Pend(15px)--mv2 Pend(10px) Bxz(bb) Py(8px) Bdends(s) Bdbs(s) Bdstarts(s) Bdstartw(1px) Bdbw(1px) Bdendw(1px) Bdc($seperatorColor) Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pstart(30px)--mv2 Pstart(25px)"><div class="D(ib) Va(m) Ell Mt(-3px) W(200px)--mv2 W(185px) undefined" title="Operating Revenue"><span class="Va(m)">Operating Revenue</span></div><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>94,028,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>81,462,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>53,823,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>31,536,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>24,578,000</span></div></div><div></div></div></div>

soup = BeautifulSoup(row_html, 'html.parser')
divs = soup.find_all('div', class_=lambda x: x and 'fi-row' in x.split())
for div in divs:
    row=[]
    elements = div.find_all(lambda tag: tag.name == 'span' and 
                             (tag.get('class') == ['Va(m)']))
    for element in elements:
        row.append(element.text)
        # print(element.text)
    elements = div.find_all(lambda tag: tag.get('data-test') == 'fin-col')
    for element in elements:
        row.append(element.text)
        # print(element.text)
        
    csv_data.append(row)
        
print(csv_data)
with open(f'{stock}_income_statement.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)
    

Breakdown
ttm
12/30/2022
12/30/2021
12/30/2020
12/30/2019
[['Breakdown', 'ttm', '12/30/2022', '12/30/2021', '12/30/2020', '12/30/2019'], ['Total Revenue', '94,028,000', '81,462,000', '53,823,000', '31,536,000', '24,578,000'], ['Cost of Revenue', '73,825,000', '60,609,000', '40,217,000', '24,906,000', '20,509,000'], ['Gross Profit', '20,203,000', '20,853,000', '13,606,000', '6,630,000', '4,069,000'], ['Operating Expense', '7,517,000', '7,021,000', '7,110,000', '4,636,000', '3,989,000'], ['Operating Income', '12,686,000', '13,832,000', '6,496,000', '1,994,000', '80,000'], ['Net Non Operating Interest Income Expense', '551,000', '106,000', '-315,000', '-718,000', '-641,000'], ['Other Income Expense', '119,000', '-219,000', '162,000', '-122,000', '-104,000'], ['Pretax Income', '13,356,000', '13,719,000', '6,343,000', '1,154,000', '-665,000'], ['Tax Provision', '1,165,000', '1,132,000', '699,000', '292,000', '110,000'], ['Net Income Common Stockholders', '12,235,000', '12,583,000', '5,519,0

### Balance Sheet

In [112]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[1]/div[1]/div/div[2]/a').click()

In [113]:
# Press expand all button 
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()


create csv

In [114]:
table_header = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div')
table_html = table_header.get_attribute('innerHTML')
soup = BeautifulSoup(table_html, 'html.parser')
spans = soup.find_all('span')

csv_data = []
row = []
for span in spans:
    row.append(span.text)
    print(span.text)

csv_data.append(row)


row = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div')
row_html = row.get_attribute('innerHTML')
# <div class="D(tbr) fi-row Bgc($hoverBgColor):h"><div class="D(tbc) Ta(start) Pend(15px)--mv2 Pend(10px) Bxz(bb) Py(8px) Bdends(s) Bdbs(s) Bdstarts(s) Bdstartw(1px) Bdbw(1px) Bdendw(1px) Bdc($seperatorColor) Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pstart(15px)--mv2 Pstart(10px)"><div class="D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined" title="Total Revenue"><button aria-label="Total Revenue" class="P(0) M(0) Va(m) Bd(0) Fz(s) Mend(2px) tgglBtn"><svg class="H(16px) Fill($primaryColor) Stk($primaryColor) tgglBtn:h_Fill($linkColor) tgglBtn:h_Stk($linkColor) Cur(p)" width="16" style="stroke-width:0;vertical-align:bottom" height="16" viewBox="0 0 48 48" data-icon="caret-down"><path d="M24.21 33.173l12.727-12.728c.78-.78.78-2.048 0-2.828-.78-.78-2.047-.78-2.828 0l-9.9 9.9-9.9-9.9c-.78-.78-2.047-.78-2.827 0-.78.78-.78 2.047 0 2.828L24.21 33.173z"></path></svg></button><span class="Va(m)">Total Revenue</span></div><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>94,028,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>81,462,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>53,823,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>31,536,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>24,578,000</span></div></div><div><div class="rw-expnded" data-test="fin-row"><div class="D(tbr) fi-row Bgc($hoverBgColor):h"><div class="D(tbc) Ta(start) Pend(15px)--mv2 Pend(10px) Bxz(bb) Py(8px) Bdends(s) Bdbs(s) Bdstarts(s) Bdstartw(1px) Bdbw(1px) Bdendw(1px) Bdc($seperatorColor) Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pstart(30px)--mv2 Pstart(25px)"><div class="D(ib) Va(m) Ell Mt(-3px) W(200px)--mv2 W(185px) undefined" title="Operating Revenue"><span class="Va(m)">Operating Revenue</span></div><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>94,028,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>81,462,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>53,823,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>31,536,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>24,578,000</span></div></div><div></div></div></div>

soup = BeautifulSoup(row_html, 'html.parser')
divs = soup.find_all('div', class_=lambda x: x and 'fi-row' in x.split())
for div in divs:
    row=[]
    elements = div.find_all(lambda tag: tag.name == 'span' and 
                             (tag.get('class') == ['Va(m)']))
    for element in elements:
        row.append(element.text)
        # print(element.text)
    elements = div.find_all(lambda tag: tag.get('data-test') == 'fin-col')
    for element in elements:
        row.append(element.text)
        # print(element.text)
        
    csv_data.append(row)
        
print(csv_data)
with open(f'{stock}_balance_sheet.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)
    

Breakdown
12/30/2022
12/30/2021
12/30/2020
12/30/2019
[['Breakdown', '12/30/2022', '12/30/2021', '12/30/2020', '12/30/2019'], ['Total Assets', '82,338,000', '62,131,000', '52,148,000', '34,309,000'], ['Current Assets', '40,917,000', '27,100,000', '26,717,000', '12,103,000'], ['Cash, Cash Equivalents & Short Term Investments', '22,185,000', '17,707,000', '19,384,000', '6,268,000'], ['Cash And Cash Equivalents', '22,185,000', '17,576,000', '19,384,000', '6,268,000'], ['Other Short Term Investments', '5,932,000', '131,000', '-', '-'], ['Receivables', '2,952,000', '1,913,000', '1,886,000', '1,324,000'], ['Accounts receivable', '2,952,000', '1,913,000', '1,886,000', '1,324,000'], ['Inventory', '12,839,000', '5,757,000', '4,101,000', '3,552,000'], ['Raw Materials', '6,137,000', '2,816,000', '1,508,000', '1,428,000'], ['Work in Process', '2,385,000', '1,089,000', '493,000', '362,000'], ['Finished Goods', '3,475,000', '1,277,000', '1,666,000', '1,356,000'], ['Other Inventories', '842,000', '57

### Cash Flow


In [5]:
# click on financials 
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[7]/a').click()

In [6]:
# Click on cash flow
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()

In [8]:
# click on expand 
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()


In [9]:
table_header = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div')
table_html = table_header.get_attribute('innerHTML')
soup = BeautifulSoup(table_html, 'html.parser')
spans = soup.find_all('span')

csv_data = []
row = []
for span in spans:
    row.append(span.text)
    print(span.text)

csv_data.append(row)


row = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div')
row_html = row.get_attribute('innerHTML')
# <div class="D(tbr) fi-row Bgc($hoverBgColor):h"><div class="D(tbc) Ta(start) Pend(15px)--mv2 Pend(10px) Bxz(bb) Py(8px) Bdends(s) Bdbs(s) Bdstarts(s) Bdstartw(1px) Bdbw(1px) Bdendw(1px) Bdc($seperatorColor) Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pstart(15px)--mv2 Pstart(10px)"><div class="D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined" title="Total Revenue"><button aria-label="Total Revenue" class="P(0) M(0) Va(m) Bd(0) Fz(s) Mend(2px) tgglBtn"><svg class="H(16px) Fill($primaryColor) Stk($primaryColor) tgglBtn:h_Fill($linkColor) tgglBtn:h_Stk($linkColor) Cur(p)" width="16" style="stroke-width:0;vertical-align:bottom" height="16" viewBox="0 0 48 48" data-icon="caret-down"><path d="M24.21 33.173l12.727-12.728c.78-.78.78-2.048 0-2.828-.78-.78-2.047-.78-2.828 0l-9.9 9.9-9.9-9.9c-.78-.78-2.047-.78-2.827 0-.78.78-.78 2.047 0 2.828L24.21 33.173z"></path></svg></button><span class="Va(m)">Total Revenue</span></div><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>94,028,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>81,462,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>53,823,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>31,536,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>24,578,000</span></div></div><div><div class="rw-expnded" data-test="fin-row"><div class="D(tbr) fi-row Bgc($hoverBgColor):h"><div class="D(tbc) Ta(start) Pend(15px)--mv2 Pend(10px) Bxz(bb) Py(8px) Bdends(s) Bdbs(s) Bdstarts(s) Bdstartw(1px) Bdbw(1px) Bdendw(1px) Bdc($seperatorColor) Pos(st) Start(0) Bgc($lv2BgColor) fi-row:h_Bgc($hoverBgColor) Pstart(30px)--mv2 Pstart(25px)"><div class="D(ib) Va(m) Ell Mt(-3px) W(200px)--mv2 W(185px) undefined" title="Operating Revenue"><span class="Va(m)">Operating Revenue</span></div><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>94,028,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>81,462,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>53,823,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)" data-test="fin-col"><span>31,536,000</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)" data-test="fin-col"><span>24,578,000</span></div></div><div></div></div></div>

soup = BeautifulSoup(row_html, 'html.parser')
divs = soup.find_all('div', class_=lambda x: x and 'fi-row' in x.split())
for div in divs:
    row=[]
    elements = div.find_all(lambda tag: tag.name == 'span' and 
                             (tag.get('class') == ['Va(m)']))
    for element in elements:
        row.append(element.text)
        # print(element.text)
    elements = div.find_all(lambda tag: tag.get('data-test') == 'fin-col')
    for element in elements:
        row.append(element.text)
        # print(element.text)
        
    csv_data.append(row)
        
print(csv_data)
with open(f'{stock}_cash_flow.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)
    

Breakdown
ttm
12/30/2022
12/30/2021
12/30/2020
12/30/2019
[['Breakdown', 'ttm', '12/30/2022', '12/30/2021', '12/30/2020', '12/30/2019'], ['Operating Cash Flow', '13,956,000', '14,724,000', '11,497,000', '5,943,000', '2,405,000'], ['Cash Flow from Continuing Operating Activities', '13,956,000', '14,724,000', '11,497,000', '5,943,000', '2,405,000'], ['Net Income from Continuing Operations', '12,191,000', '12,587,000', '5,644,000', '862,000', '-775,000'], ['Operating Gains Losses', '-116,000', '221,000', '-82,000', '231,000', '98,000'], ['Gain Loss On Sale of PPE', '34,000', '140,000', '-27,000', '117,000', '146,000'], ['Net Foreign Currency Exchange Gain Loss', '-150,000', '81,000', '-55,000', '114,000', '-48,000'], ['Depreciation Amortization Depletion', '4,145,000', '3,747,000', '2,911,000', '2,322,000', '2,154,000'], ['Depreciation & amortization', '4,145,000', '3,747,000', '2,911,000', '2,322,000', '2,154,000'], ['Depreciation', '4,145,000', '3,747,000', '2,911,000', '-', '-'], ['Ass